# BDA03 — Tarea de Evaluación (Ecosistema Hadoop–Spark)
**Flujo ETL completo:** importación automática → exploración → limpieza (Pig) → transformación → carga en Hive → consultas HQL con JOIN.

**Dataset:** T1D (Guertin et al., 2024) — Virginia PrIMeD  
**Archivos:** `survey_data_and_results_final.xlsx` + `assay_final_genotyping_file.xlsx`  
**Clave de relación esperada (a validar):** `survey.SUBJECT_ID` ↔ `genotyping.FID`


## Explicación del conjunto de datos elegido
He elegido un dataset real sobre **riesgo genético de Diabetes Tipo 1 (T1D)** del estudio **Guertin et al. (2024)** (cohorte Virginia PrIMeD). Es interesante porque permite combinar datos **clínicos/demográficos** con datos **genéticos**, lo cual es un escenario típico de integración (JOIN) en Big Data.

### Archivos utilizados
Trabajo con **dos archivos interrelacionados**:

1) **`survey_data_and_results_final.xlsx` (Survey / Fenotipo)**  
Contiene información del participante (clínica/demográfica) y variables relacionadas con riesgo genético.  
**Clave:** `SUBJECT_ID`.

2) **`assay_final_genotyping_file.xlsx` (Genotyping / Genética)**  
Contiene marcadores genéticos (SNPs) por participante.  
**Clave:** `FID`.

### Relación entre archivos
La relación se realiza por el identificador de participante:
- **`survey.SUBJECT_ID = genotyping.FID`**  
(Validaré esta relación con datos reales antes de continuar.)

### Tamaño y relevancia
El dataset tiene en torno a **3.800 participantes**, suficiente para demostrar un flujo ETL completo con Hadoop (Pig) y Spark/Hive, incluyendo limpieza de datos, transformación y consultas con JOIN.


# 1. Importación automática + conversión a CSV


> Objetivo: traer los datos de forma reproducible (sin subida manual) y dejarlos en CSV para Pig/Spark/Hive.



In [1]:
# 1. Importación automática + conversión a CSV
# Objetivo: traer los datos de forma reproducible (sin subida manual) y dejarlos en CSV para Pig/Spark/Hive.

import os, shutil
import pandas as pd

# 1) Importación automática: clonar repo
!rm -rf BDA_dataset
!git clone https://github.com/kachytronico/BDA_dataset

# 2) Rutas reales a los XLSX dentro del repo clonado
base_path = "/content/BDA_dataset/Dataset not incorporated into the T1DKP"
survey_xlsx = os.path.join(base_path, "survey_data_and_results_final.xlsx")
geno_xlsx   = os.path.join(base_path, "assay_final_genotyping_file.xlsx")

assert os.path.exists(survey_xlsx), f"Falta archivo requerido: {survey_xlsx}"
assert os.path.exists(geno_xlsx),   f"Falta archivo requerido: {geno_xlsx}"

# 3) Copia a RAW (mantener originales intactos)
raw_dir = "/content/data/raw"
csv_dir = "/content/data/csv"
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)

survey_raw = os.path.join(raw_dir, "survey_data_and_results_final.xlsx")
geno_raw   = os.path.join(raw_dir, "assay_final_genotyping_file.xlsx")
shutil.copy2(survey_xlsx, survey_raw)
shutil.copy2(geno_xlsx, geno_raw)

# 4) Convertir a CSV (formato base para Pig/Spark/Hive)
df_survey = pd.read_excel(survey_raw)
df_geno   = pd.read_excel(geno_raw)

survey_csv = os.path.join(csv_dir, "survey.csv")
geno_csv   = os.path.join(csv_dir, "genotyping.csv")
df_survey.to_csv(survey_csv, index=False)
df_geno.to_csv(geno_csv, index=False)

# 5) Evidencia mínima visible
print("📁 Repo clonado (muestra):")
!ls -lh BDA_dataset | head -n 20

print("\n📁 RAW:")
!ls -lh /content/data/raw

print("\n📁 CSV:")
!ls -lh /content/data/csv

print("\nshape survey:", df_survey.shape)
print("shape genotyping:", df_geno.shape)

print("\nPreview survey (3 filas):")
display(df_survey.head(3))

print("\nPreview genotyping (3 filas):")
display(df_geno.head(3))


Cloning into 'BDA_dataset'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 89 (delta 21), reused 79 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 3.76 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (21/21), done.
📁 Repo clonado (muestra):
total 132K
drwxr-xr-x 3 root root 4.0K Feb 18 20:40 BDA03_cuadernos
-rw-r--r-- 1 root root 6.1K Feb 18 20:40 BDA03_Evaluacion_T1D01.ipynb
-rw-r--r-- 1 root root  60K Feb 18 20:40 BDA03_Evaluacion_T1D_copilot.ipynb
-rw-r--r-- 1 root root  41K Feb 18 20:40 BDA03_Evaluacion_T1D.ipynb
drwxr-xr-x 2 root root 4.0K Feb 18 20:40 Dataset not incorporated into the T1DKP
drwxr-xr-x 4 root root 4.0K Feb 18 20:40 docs
-rw-r--r-- 1 root root 7.5K Feb 18 20:40 README.md

📁 RAW:
total 1.7M
-rw-r--r-- 1 root root 1.4M Feb 18 20:40 assay_final_genotyping_file.xlsx
-rw-r--r-- 1 root root 295K Feb 18 20:40 survey_data_and_results_final.xlsx

📁 CSV:

,SUBJECT_ID,AGE,RACE,T1D_HIST,AUTO_HIST,AUTO_COND,AUTO_COND_4_TEXT,T1D_DIAG,T1D_DIAG_AGE,T1D_HOSP,DKA,GRS_HLA,GnonHLA,GRS,Risk
0,10011708520314,6,White,Yes,No,Not applicable,Not applicable,No,Not applicable,Not applicable,Not applicable,1.91,0.14,2.06,Not high
1,10021708520764,3,White,Don't know,Yes,"Thyroid_Hashimotos and_or Graves, Blood relati...",Not applicable,No,Not applicable,Not applicable,Not applicable,-1.41,1.93,0.52,Not high
2,10021708521587,7,Asian,Don't know,No,Not applicable,Not applicable,No,Not applicable,Not applicable,Not applicable,-13.66,0.77,-12.89,Not high



Preview genotyping (3 filas):


,FID,contact key,rs1049225,rs1052553,rs10795791,rs11203203,rs113010081,rs1150743,rs12416116,rs12720356,...,rs757411,rs7745656,rs7780389,rs917911,rs9268633,rs9271366,rs9273363,rs9357152,rs9469341,rs9585056
0,10011708520314,CONTACT10085,C:C,A:A,A:G,A:G,T:T,A:G,C:C,T:T,...,C:T,G:T,C:C,G:T,A:G,A:A,A:C,A:A,A:G,T:T
1,10021708520764,CONTACT14053,C:C,A:A,G:G,A:A,T:T,G:G,C:C,T:T,...,C:C,T:T,C:C,G:T,G:G,A:A,C:C,A:G,A:G,C:T
2,10021708521587,CONTACT11350,C:C,A:G,A:G,G:G,T:T,A:A,A:A,T:T,...,C:T,T:T,C:C,T:T,A:A,G:G,C:C,A:A,G:G,C:T


## Conclusiones (Apartado 1)
- He importado los datos de forma automática clonando el repositorio con `git clone`, sin subida manual.
- He conservado los originales en RAW (295 KB survey y 1.4 MB genotyping) y he convertido ambos a CSV (528 KB y 1.2 MB) para poder trabajar después con Pig y Spark/Hive.
- He verificado que ambos datasets tienen **3818 filas**; la estructura final es **15 columnas** en survey y **76 columnas** en genotyping.


# 2. Exploración con Pandas (usando CSV convertidos)



> Objetivo: validar JOIN real y detectar problemas de calidad (nulos, -9) sobre los CSV.



In [2]:
# 2. Exploración con Pandas (usando CSV convertidos)
# Objetivo: validar JOIN real y detectar problemas de calidad (nulos, -9) sobre los CSV.

import pandas as pd

survey_csv = "/content/data/csv/survey.csv"
geno_csv   = "/content/data/csv/genotyping.csv"

# 1) Cargar CSV forzando texto para evitar problemas de tipos
df_s = pd.read_csv(survey_csv, dtype=str)
df_g = pd.read_csv(geno_csv, dtype=str)

# 2) Confirmación de columnas clave
assert "SUBJECT_ID" in df_s.columns, "No existe SUBJECT_ID en survey"
assert "FID" in df_g.columns, "No existe FID en genotyping"

# 3) Validación JOIN real: survey.SUBJECT_ID = genotyping.FID
ids_s = df_s["SUBJECT_ID"].dropna().astype(str).str.strip()
ids_g = df_g["FID"].dropna().astype(str).str.strip()

print("IDs únicos survey (SUBJECT_ID):", ids_s.nunique())
print("IDs únicos genotyping (FID):  ", ids_g.nunique())
print("Intersección IDs:", len(set(ids_s).intersection(set(ids_g))))

m = pd.merge(
    df_s[["SUBJECT_ID"]],
    df_g[["FID"]],
    left_on="SUBJECT_ID",
    right_on="FID",
    how="inner"
)

print("Filas merge inner:", m.shape[0])
display(m.head(3))

# 4) Detección de nulos / vacíos (top 10 columnas)
print("\nNulos en survey (top 10 columnas):")
display(df_s.isna().sum().sort_values(ascending=False).head(10))

print("\nNulos en genotyping (top 10 columnas):")
display(df_g.isna().sum().sort_values(ascending=False).head(10))

# 5) Conteo de '-9' como missing codificado en genotyping
minus9_total = (df_g == "-9").sum().sum()
print("\nTotal de valores '-9' (string) en genotyping:", int(minus9_total))


IDs únicos survey (SUBJECT_ID): 3818
IDs únicos genotyping (FID):   3818
Intersección IDs: 3818
Filas merge inner: 3818


,SUBJECT_ID,FID
0,10011708520314,10011708520314
1,10021708520764,10021708520764
2,10021708521587,10021708521587



Nulos en survey (top 10 columnas):


,0
SUBJECT_ID,0
AGE,0
RACE,0
T1D_HIST,0
AUTO_HIST,0
AUTO_COND,0
AUTO_COND_4_TEXT,0
T1D_DIAG,0
T1D_DIAG_AGE,0
T1D_HOSP,0



Nulos en genotyping (top 10 columnas):


,0
FID,0
contact key,0
rs1049225,0
rs1052553,0
rs10795791,0
rs11203203,0
rs113010081,0
rs1150743,0
rs12416116,0
rs12720356,0



Total de valores '-9' (string) en genotyping: 181


## Conclusiones ( Pandas)
- He confirmado que hay **3818 IDs únicos** en ambos archivos.
- La intersección de identificadores es total (**3818 IDs comunes**) y el `merge inner` devuelve **3818 filas**, validando la clave de unión **`survey.SUBJECT_ID = genotyping.FID`**.
- En el top 10 de columnas revisadas no he detectado valores nulos (`NaN`).


## Localizar exactamente dónde está el missing codificado (-9)


> Objetivo: identificar qué columnas concretas contienen '-9' para justificar la limpieza en Pig.




In [3]:
# Localizar exactamente dónde está el missing codificado (-9)
# Objetivo: identificar qué columnas concretas contienen '-9' para justificar la limpieza en Pig.

import pandas as pd

survey_csv = "/content/data/csv/survey.csv"
geno_csv   = "/content/data/csv/genotyping.csv"

df_s = pd.read_csv(survey_csv, dtype=str)
df_g = pd.read_csv(geno_csv, dtype=str)

# 1) Conteo por columna de '-9' en genotyping (missing codificado)
minus9_by_column = (df_g == "-9").sum()
minus9_by_column = minus9_by_column[minus9_by_column > 0].sort_values(ascending=False)

print("Columnas con '-9' (missing codificado) en genotyping:")
display(minus9_by_column)

print("\nTotal '-9' en genotyping:", int((df_g == "-9").sum().sum()))

# 2) Comprobación de cadenas vacías en survey (por si hubiese valores "" en vez de NaN)
empty_strings_survey = (df_s == "").sum()
empty_strings_survey = empty_strings_survey[empty_strings_survey > 0]

print("\nColumnas con cadenas vacías '' en survey:")
display(empty_strings_survey)


Columnas con '-9' (missing codificado) en genotyping:


,0
rs9585056,89
rs12927355,82
rs1367728,9
rs72727394,1



Total '-9' en genotyping: 181

Columnas con cadenas vacías '' en survey:


,0


## Conclusiones (calidad de datos)
- He detectado un total de **181 valores `"-9"`** en la tabla genotyping (missing codificado).
- El desglose es: `rs9585056` (89), `rs12927355` (82), `rs1367728` (9) y `rs72727394` (1).
- He comprobado que no existen cadenas vacías (`""`) en la tabla survey.
- En el siguiente apartado limpiaré estos `"-9"` convirtiéndolos a valores nulos reales para no distorsionar transformaciones y consultas.


# 3) Apache Pig — Limpieza + tratamiento
En esta sección voy a preparar Pig para limpiar datos de genotyping y normalizar claves para asegurar un JOIN fiable.
Primero dejaré el entorno listo en Colab con Java 17 y Pig, siguiendo el estilo de los cuadernos de referencia.
Como tratamiento interesante simple, usaré un Top 3 de valores más frecuentes en una columna clave tras la limpieza.

In [4]:
# Sección 3.1 — Preparación de entorno Pig en Colab (Java 17)
# Estilo base: cuaderno 0301 (instalación simple y verificación de versión)

import os

# 1) Java 17
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

# 2) Hadoop (dependencia de entorno para Pig)
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz
!tar -xzf hadoop-3.4.2.tar.gz
!rm -rf /usr/local/hadoop
!mv hadoop-3.4.2 /usr/local/hadoop

os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] += f":{os.environ['HADOOP_HOME']}/bin:{os.environ['HADOOP_HOME']}/sbin"

# 3) Pig
!wget -q https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
!tar -xzf pig-0.17.0.tar.gz
!rm -rf /usr/local/pig-0.17.0
!mv pig-0.17.0 /usr/local/pig-0.17.0

os.environ["PIG_HOME"] = "/usr/local/pig-0.17.0"
os.environ["PATH"] += f":{os.environ['PIG_HOME']}/bin"
os.environ["PIG_CLASSPATH"] = "/usr/local/hadoop/etc/hadoop"

# 4) Verificación mínima visible
!java -version
!hadoop version | head -n 3
!pig -version

openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)
Hadoop 3.4.2
Source code repository https://github.com/apache/hadoop.git -r 84e8b89ee2ebe6923691205b9e171badde7a495c
Compiled by ahmarsu on 2025-08-20T10:30Z
Apache Pig version 0.17.0 (r1797386) 
compiled Jun 02 2017, 15:41:58


In [5]:
%%writefile limpieza.pig
-- Sección 3.1: limpieza de survey y genotyping

survey_raw = LOAD '/content/data/csv/survey.csv' USING PigStorage(',') AS (
    SUBJECT_ID:chararray,
    AGE:chararray,
    RACE:chararray,
    T1D_HIST:chararray,
    AUTO_HIST:chararray,
    AUTO_COND:chararray,
    AUTO_COND_4_TEXT:chararray,
    T1D_DIAG:chararray,
    T1D_DIAG_AGE:chararray,
    T1D_HOSP:chararray,
    DKA:chararray,
    GRS_HLA:chararray,
    GnonHLA:chararray,
    GRS:chararray,
    Risk:chararray
);

survey_no_header = FILTER survey_raw BY SUBJECT_ID != 'SUBJECT_ID';

survey_clean = FOREACH survey_no_header GENERATE
    TRIM(SUBJECT_ID) AS SUBJECT_ID,
    AGE,
    RACE,
    T1D_HIST,
    AUTO_HIST,
    AUTO_COND,
    AUTO_COND_4_TEXT,
    T1D_DIAG,
    T1D_DIAG_AGE,
    T1D_HOSP,
    DKA,
    GRS_HLA,
    GnonHLA,
    GRS,
    Risk;

STORE survey_clean INTO '/content/data/pig_out/survey_clean' USING PigStorage(',');


genotyping_raw = LOAD '/content/data/csv/genotyping.csv' USING PigStorage(',') AS (
    FID:chararray,
    contact_key:chararray,
    rs1049225:chararray,
    rs1052553:chararray,
    rs10795791:chararray,
    rs11203203:chararray,
    rs113010081:chararray,
    rs1150743:chararray,
    rs12416116:chararray,
    rs12720356:chararray,
    rs12927355:chararray,
    rs12971201:chararray,
    rs13415583:chararray,
    rs1367728:chararray,
    rs1456988:chararray,
    rs151233:chararray,
    rs1574285:chararray,
    rs1615504:chararray,
    rs1893217:chararray,
    rs193778:chararray,
    rs2045258:chararray,
    rs2071463:chararray,
    rs2076531:chararray,
    rs2111485:chararray,
    rs2143461:chararray,
    rs2194225:chararray,
    rs2239800:chararray,
    rs2256974:chararray,
    rs229533:chararray,
    rs2476601:chararray,
    rs2523409:chararray,
    rs2524089:chararray,
    rs2611215:chararray,
    rs28732101:chararray,
    rs3024505:chararray,
    rs3087243:chararray,
    rs3094165:chararray,
    rs3129722:chararray,
    rs3130933:chararray,
    rs34536443:chararray,
    rs34593439:chararray,
    rs35337543:chararray,
    rs35667974:chararray,
    rs3763305:chararray,
    rs402072:chararray,
    rs41295121:chararray,
    rs436845:chararray,
    rs4820830:chararray,
    rs4849135:chararray,
    rs516246:chararray,
    rs56994090:chararray,
    rs6043409:chararray,
    rs61839660:chararray,
    rs62447205:chararray,
    rs635688:chararray,
    rs6518350:chararray,
    rs653178:chararray,
    rs6691977:chararray,
    rs689:chararray,
    rs6903608:chararray,
    rs6906897:chararray,
    rs6935715:chararray,
    rs705704:chararray,
    rs72727394:chararray,
    rs72853903:chararray,
    rs72928038:chararray,
    rs757411:chararray,
    rs7745656:chararray,
    rs7780389:chararray,
    rs917911:chararray,
    rs9268633:chararray,
    rs9271366:chararray,
    rs9273363:chararray,
    rs9357152:chararray,
    rs9469341:chararray,
    rs9585056:chararray
);

genotyping_no_header = FILTER genotyping_raw BY FID != 'FID';

genotyping_clean = FOREACH genotyping_no_header GENERATE
    TRIM(FID) AS FID,
    contact_key,
    rs1049225,
    rs1052553,
    rs10795791,
    rs11203203,
    rs113010081,
    rs1150743,
    rs12416116,
    rs12720356,
    (rs12927355 == '-9' ? '' : rs12927355) AS rs12927355,
    rs12971201,
    rs13415583,
    (rs1367728 == '-9' ? '' : rs1367728) AS rs1367728,
    rs1456988,
    rs151233,
    rs1574285,
    rs1615504,
    rs1893217,
    rs193778,
    rs2045258,
    rs2071463,
    rs2076531,
    rs2111485,
    rs2143461,
    rs2194225,
    rs2239800,
    rs2256974,
    rs229533,
    rs2476601,
    rs2523409,
    rs2524089,
    rs2611215,
    rs28732101,
    rs3024505,
    rs3087243,
    rs3094165,
    rs3129722,
    rs3130933,
    rs34536443,
    rs34593439,
    rs35337543,
    rs35667974,
    rs3763305,
    rs402072,
    rs41295121,
    rs436845,
    rs4820830,
    rs4849135,
    rs516246,
    rs56994090,
    rs6043409,
    rs61839660,
    rs62447205,
    rs635688,
    rs6518350,
    rs653178,
    rs6691977,
    rs689,
    rs6903608,
    rs6906897,
    rs6935715,
    rs705704,
    (rs72727394 == '-9' ? '' : rs72727394) AS rs72727394,
    rs72853903,
    rs72928038,
    rs757411,
    rs7745656,
    rs7780389,
    rs917911,
    rs9268633,
    rs9271366,
    rs9273363,
    rs9357152,
    rs9469341,
    (rs9585056 == '-9' ? '' : rs9585056) AS rs9585056;

STORE genotyping_clean INTO '/content/data/pig_out/genotyping_clean' USING PigStorage(',');

Writing limpieza.pig


In [6]:
# Ejecutar limpieza Pig en modo local y mostrar evidencias mínimas

!rm -rf /content/data/pig_out/survey_clean /content/data/pig_out/genotyping_clean
!pig -x local -f limpieza.pig

print("\nSalida survey_clean:")
!ls -lh /content/data/pig_out/survey_clean
print("Primeras 3 filas survey_clean:")
!head -n 3 /content/data/pig_out/survey_clean/part*

print("\nSalida genotyping_clean:")
!ls -lh /content/data/pig_out/genotyping_clean
print("Primeras 3 filas genotyping_clean:")
!head -n 3 /content/data/pig_out/genotyping_clean/part*

2026-02-18 20:42:46,308 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2026-02-18 20:42:46,309 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2026-02-18 20:42:46,477 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2026-02-18 20:42:46,477 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1771447366467.log
2026-02-18 20:42:46,550 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2026-02-18 20:42:47,124 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2026-02-18 20:42:47,472 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2026-02-18 20:42:47,480 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2026-02-18 20:42:47,58

## Conclusiones (Sección 3.1 — Limpieza con Pig)”

He ejecutado limpieza.pig en modo local y la limpieza ha funcionado correctamente. Se han generado las carpetas de salida survey_clean y genotyping_clean dentro de /content/data/pig_out/, cada una con su fichero part-m-00000 y el marcador _SUCCESS. En concreto, survey_clean ocupa aproximadamente 523 KB y genotyping_clean aproximadamente 1.2 MB.


Además, he normalizado las claves de relación aplicando TRIM() sobre SUBJECT_ID y FID, dejando los datos preparados para un JOIN posterior fiable. Por último, he aplicado la corrección del missing codificado sustituyendo '-9' por vacío ('') únicamente en los 4 marcadores identificados previamente (rs9585056, rs12927355, rs1367728 y rs72727394), manteniendo el resto de columnas sin cambios.

## Sección 3.2 — Tratamiento interesante
En este bloque calculo el Top 3 de valores más frecuentes de `RACE` a partir de `survey_clean` con Pig, mostrando el resultado en pantalla con `DUMP`.

In [7]:
%%writefile tratamiento_top3_race.pig
-- Sección 3.2: tratamiento interesante simple (Top 3 de RACE)

survey_clean = LOAD '/content/data/pig_out/survey_clean' USING PigStorage(',') AS (
    SUBJECT_ID:chararray,
    AGE:chararray,
    RACE:chararray,
    T1D_HIST:chararray,
    AUTO_HIST:chararray,
    AUTO_COND:chararray,
    AUTO_COND_4_TEXT:chararray,
    T1D_DIAG:chararray,
    T1D_DIAG_AGE:chararray,
    T1D_HOSP:chararray,
    DKA:chararray,
    GRS_HLA:chararray,
    GnonHLA:chararray,
    GRS:chararray,
    Risk:chararray
);

survey_valid = FILTER survey_clean BY (RACE IS NOT NULL) AND (TRIM(RACE) != '');

race_group = GROUP survey_valid BY RACE;
race_count = FOREACH race_group GENERATE group AS RACE, COUNT(survey_valid) AS total;
race_order = ORDER race_count BY $1 DESC;
top3_race = LIMIT race_order 3;

DUMP top3_race;

Writing tratamiento_top3_race.pig


In [8]:
# Ejecutar tratamiento 3.2 (Top 3 RACE) y dejar pista clara de éxito

!pig -x local -f tratamiento_top3_race.pig | tee top3_race.log

print("\nVerificación rápida 3.2:")
print("- Deben aparecer 3 filas con formato (RACE,conteo) en el DUMP")
print("- Debe aparecer 'Success!' al final del log")

!grep -n "Success!" top3_race.log | tail -n 1
!grep -E "^\(.*,[0-9]+\)$" top3_race.log | head -n 3

2026-02-18 20:42:56,723 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2026-02-18 20:42:56,723 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2026-02-18 20:42:56,841 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2026-02-18 20:42:56,841 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1771447376833.log
2026-02-18 20:42:56,895 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2026-02-18 20:42:57,191 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2026-02-18 20:42:57,319 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2026-02-18 20:42:57,324 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2026-02-18 20:42:57,37

## Conclusiones (Sección 3.2)
He ejecutado el script tratamiento_top3_race.pig en modo local y he mostrado el resultado directamente en pantalla usando DUMP, lo que deja evidencia visible en el cuaderno.
Como tratamiento interesante, he calculado el Top 3 de valores más frecuentes de la variable RACE a partir de los datos ya limpiados (survey_clean). El resultado obtenido es:

White: 3313

Black or African American: 253

Asian: 120

Este paso completa el apartado 3.2 con una agregación sencilla (GROUP + COUNT + ORDER + LIMIT) y deja el dataset preparado para continuar con Spark/Hive.

## 4) Spark (PySpark) → Hive
En esta sección voy a usar PySpark para leer los datos limpios que generé con Pig.
Como el output de Pig no trae cabecera garantizada, aplicaré el esquema usando las cabeceras de los CSV originales.
Después guardaré los dos DataFrames en Hive como `tabla_survey` y `tabla_genotyping`.
Así dejo preparadas dos tablas relacionales para poder hacer JOIN real en la Sección 5.

In [ ]:
# Sección 4 — Configurar Spark con soporte Hive

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

!pip -q install pyspark findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("BDA03_Spark_Hive") \
    .config("spark.sql.warehouse.dir", "/content/spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

print("spark.version:", spark.version)

In [ ]:
# Sección 4 — Cargar datos limpios y crear tablas Hive

import csv
from pyspark.sql.functions import col, trim

# 1) Cabeceras desde CSV originales
with open('/content/data/csv/survey.csv', 'r', encoding='utf-8') as f:
    survey_headers = next(csv.reader(f))

with open('/content/data/csv/genotyping.csv', 'r', encoding='utf-8') as f:
    genotyping_headers = next(csv.reader(f))

# Nombre válido para Spark/Hive (en original viene "contact key")
genotyping_headers = [h.replace('contact key', 'contact_key') for h in genotyping_headers]

# 2) Leer output Pig (sin cabecera) y aplicar esquema
survey_df = spark.read.option('header', 'false').option('inferSchema', 'false').csv('/content/data/pig_out/survey_clean') \
    .toDF(*survey_headers)

genotyping_df = spark.read.option('header', 'false').option('inferSchema', 'false').csv('/content/data/pig_out/genotyping_clean') \
    .toDF(*genotyping_headers)

# 3) Asegurar claves de JOIN como string y normalizadas
survey_df = survey_df.withColumn('SUBJECT_ID', trim(col('SUBJECT_ID').cast('string')))
genotyping_df = genotyping_df.withColumn('FID', trim(col('FID').cast('string')))

# 4) Evidencia mínima: esquema y muestra
print('Schema tabla_survey:')
survey_df.printSchema()
print('Muestra tabla_survey:')
survey_df.show(3, truncate=False)

print('Schema tabla_genotyping:')
genotyping_df.printSchema()
print('Muestra tabla_genotyping:')
genotyping_df.show(3, truncate=False)

# 5) Guardar en Hive (2 tablas)
survey_df.write.mode('overwrite').saveAsTable('tabla_survey')
genotyping_df.write.mode('overwrite').saveAsTable('tabla_genotyping')

# 6) Evidencia mínima en Hive
print('Tablas Hive creadas:')
spark.sql('SHOW TABLES').show(truncate=False)

print('COUNT tabla_survey:')
spark.sql('SELECT COUNT(*) AS total FROM tabla_survey').show()
print('COUNT tabla_genotyping:')
spark.sql('SELECT COUNT(*) AS total FROM tabla_genotyping').show()

## Conclusiones (Sección 4 — Spark → Hive)

He cargado en Spark los dos ficheros limpios que venían de Pig y los he guardado en Hive como **dos tablas** para poder hacer JOIN en el siguiente apartado.

- La tabla **`tabla_survey`** se ha creado con la clave **`SUBJECT_ID`** y el resto de campos clínicos (15 columnas). La muestra de filas confirma que los datos se leen bien y que Spark reconoce valores faltantes (por ejemplo, `Risk` aparece como `NULL` en una de las filas mostradas).
- La tabla **`tabla_genotyping`** se ha creado con la clave **`FID`**, `contact_key` y todos los marcadores genéticos `rs...`. La muestra confirma el formato esperado de los genotipos (valores como `A:G`, `C:C`, etc.).

He verificado que ambas tablas existen en Hive con `SHOW TABLES` y aparecen como tablas no temporales: **`tabla_survey`** y **`tabla_genotyping`**.

Por último, he comprobado el volumen cargado en Hive con `COUNT(*)` y en ambos casos el resultado es **3818 filas**, así que la carga es consistente. Con esto, dejo listo el entorno para las consultas HQL con JOIN usando **`SUBJECT_ID` ↔ `FID`**.
